# GF(epfl1)

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#from ipywidgets import *
import cv2
import time
#import kernels
import skimage
#from skimage import io as skimage_io
import mrcfile
import logging
from collections import namedtuple
import information_theory
from denoising.volume.gaussian import Monochrome_Denoising as GF

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
Args = namedtuple("args", ["input", "output"])
args = Args("noisy.mrc", "denoised.mrc")
print(args)

In [ ]:
#noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
Z_dim = noisy.shape[0]
Z2 = Z_dim//2

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

std_dev = 2.0
sigma = np.array([std_dev, std_dev, std_dev])
kernel = [None]*3
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
kernel[2] = get_gaussian_kernel(sigma[2])

In [ ]:
denoiser = GF(logger)
denoised = denoiser.filter(noisy, kernel)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(denoised[Z2], cmap="gray")
axes[1].imshow(noisy[Z2], cmap="gray")
plt.show()

In [ ]:
with mrcfile.new(args.output, overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path=args.output,
    drive_file_name=f'epfl1__GF.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
def read_MRC(file_path):
    return mrcfile.read(file_path)

In [ ]:
denoised = read_MRC(args.output)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("FlowDenoising")
plt.imshow(denoised[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[7][::-1, :] - denoised[7][::-1, :], cmap="gray")

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 300:, 600:], cmap="gray")
plt.savefig("denoised.pdf", bbox_inches='tight')

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.pdf',
    drive_file_name='epfl1__GF.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)